In [1]:
%load_ext autoreload
%autoreload 2
from hypopredict.fusion.ml_preproc import MLPreprocessor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')

mlp = MLPreprocessor()
path = '/Users/alexxela/code/hypopredict/data/processed/fusion_2/TRAIN_ml_prepped_cv_splits_20251217_193254.pkl'
loaded_dict = MLPreprocessor().load_prepped_splits(path)
splits_prepped = loaded_dict['splits_prepped']

# carefully drop na values from each split
for SPLIT_INDEX in range(len(splits_prepped)):
    splits_prepped[SPLIT_INDEX][0]['target'] = splits_prepped[SPLIT_INDEX][1].copy()
    splits_prepped[SPLIT_INDEX][0].dropna(inplace=True)
    splits_prepped[SPLIT_INDEX] = (splits_prepped[SPLIT_INDEX][0].drop('target', axis=1),
                                   splits_prepped[SPLIT_INDEX][0]['target'].to_numpy())
    print(f"Split {SPLIT_INDEX} after dropping NA: X shape {splits_prepped[SPLIT_INDEX][0].shape}, y shape {splits_prepped[SPLIT_INDEX][1].shape}")

# instantiate CV
from hypopredict.cv import CrossValidator
cv = CrossValidator(splits=splits_prepped)

Split 0 after dropping NA: X shape (32409, 7), y shape (32409,)
Split 1 after dropping NA: X shape (35385, 7), y shape (35385,)
Split 2 after dropping NA: X shape (36765, 7), y shape (36765,)
Split 3 after dropping NA: X shape (30690, 7), y shape (30690,)
Split 4 after dropping NA: X shape (39135, 7), y shape (39135,)


In [7]:
# initialize XGBoost model
from xgboost import XGBClassifier
xgb = XGBClassifier(
    n_estimators=777,
    max_depth=4,
    reg_lambda=0.3,
    reg_alpha=0.05,
    learning_rate=0.1,
    eval_metric='logloss',
    random_state=17
)



from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

svm = Pipeline([
    ("scaler", StandardScaler()),
    ("svm", SVC(kernel="rbf", C=25.0, gamma="scale", probability=True))
])



# fit a KNN classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=11)



from sklearn.linear_model import LogisticRegression
#from sklearn.pipeline import Pipeline
#from sklearn.preprocessing import StandardScaler
# fusion = Pipeline([
#     ("scaler", StandardScaler()),
#     ("logreg", LogisticRegression(max_iter=15000))
# ])

#fusion = LogisticRegression(max_iter=15000)
# TODO: make it lgbm for faster training

fusion = XGBClassifier(
    n_estimators=222,
    max_depth=2,
    reg_lambda=0.4,
    reg_alpha=0.01,
    learning_rate=0.01,
    eval_metric='logloss',
    random_state=17
)


In [8]:
# training on 145000 rows at a time
cv_results = cv.validate_model_cv(models=[xgb, svm, knn, fusion],
                                    fusion=True,
                                    splits_prepped=splits_prepped,
                                    resample=True,
                                    desired_pos_ratio=0.4,
                                    reduction_factor=0.7)

Fold  1 / 5
Fold  2 / 5
Fold  3 / 5
Fold  4 / 5
Fold  5 / 5


In [9]:
cv_results

{'val_pr_aucs': [0.997,
  0.148,
  0.997,
  0.147,
  0.998,
  0.175,
  0.996,
  0.133,
  0.996,
  0.057],
 'val_ave_precisions': [0.996,
  0.138,
  0.996,
  0.133,
  0.998,
  0.154,
  0.995,
  0.13,
  0.995,
  0.044]}

In [10]:
odd_numbers = [i for i in range(1,10,2)]
val_aucs = [cv_results['val_pr_aucs'][i] for i in odd_numbers]
val_ave_precisions = [cv_results['val_ave_precisions'][i] for i in odd_numbers]
print(f"Average validation AUC-PR: {np.mean(val_aucs).round(4)}", val_aucs)
print(f"Average validation Average Precision: {np.mean(val_ave_precisions).round(4)}", val_ave_precisions)
print(f"Baseline: {np.mean([splits_prepped[i][1].mean() for i in range(len(splits_prepped))]).round(4)}")

Average validation AUC-PR: 0.132 [0.148, 0.147, 0.175, 0.133, 0.057]
Average validation Average Precision: 0.1198 [0.138, 0.133, 0.154, 0.13, 0.044]
Baseline: 0.1207
